In [3]:
import re
import csv
import pickle
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
#from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import roc_auc_score
#from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from datetime import datetime
from elasticsearch import Elasticsearch #pip install Elasticsearch if not intalled yet
import json
import calendar
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
#start processing data twitter
def processTweet(tweet):
    # process the tweets
    #Convert to lower case
    tweet = tweet.lower()
    #Convert www.* or https?://* to URL
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','URL',tweet)
    #Convert @username to AT_USER
    tweet = re.sub('@[^\s]+','AT_USER',tweet)
    #Remove additional white spaces
    tweet = re.sub('[\s]+', ' ', tweet)
    #Replace #word with word
    tweet = re.sub(r'#([^\s]+)', r'\1', tweet)
    #trim
    tweet = tweet.strip('\'"')
#     print(tweet)
#     print(' ')
    return tweet
#end

In [5]:
#initialize stopWords
stopWords = []

In [6]:
#start replaceTwoOrMore
def replaceTwoOrMore(s):
    #look for 2 or more repetitions of character and replace with the character itself
    pattern = re.compile(r"(.)\1{1,}", re.DOTALL)
    return pattern.sub(r"\1\1", s)
#end

In [7]:
#start getStopWordList
def getStopWordList(stopWordListFileName):
    #read the stopwords file and build a list
    stopWords = []
    stopWords.append('AT_USER')
    stopWords.append('URL')

    fp = open(stopWordListFileName, 'r')
    line = fp.readline()
    while line:
        word = line.strip()
        stopWords.append(word)
        line = fp.readline()
    fp.close()
    return stopWords
#end

In [8]:
#start getfeatureVector
def getFeatureVector(tweet):
    featureVector = []
    # tweet=stemmer.stem(tweet)
    #split tweet into words
    words = tweet.split()
    for w in words:
        #replace two or more with two occurrences
        w = replaceTwoOrMore(w)
        #strip punctuation
        w = w.strip('\'"?,.')
        #check if the word stats with an alphabet
        val = re.search(r"^[a-zA-Z][a-zA-Z0-9]*$", w)
        #ignore if it is a stop word
        if(w in stopWords or val is None):
            continue
        else:
           featureVector.append(w.lower())
#     print(featureVector)
#     print(' ')
    return featureVector
#end

In [9]:
#start replace singkatan/kata alay
def translator(tweet):
    j = 0
    for kata in tweet:
        with open('singkatan.txt', 'r') as csvfile:
            # Reading file as CSV with delimiter as "=", so that abbreviation are stored in row[0] and phrases in row[1]
            dataFromFile = csv.reader(csvfile, delimiter="=")
            for row in dataFromFile:
                # Check if selected word matches short forms[LHS] in text file.
                if kata == row[0]:
                    # If match found replace it with its appropriate phrase in text file.
                    tweet[j] = row[1]
            csvfile.close()
        j = j + 1
#     print(' '.join(tweet))
#     print('')
    return tweet    
#end process

In [11]:
#Read the tweets one by one and process it
df = pd.read_csv('jok.csv', delimiter=',', index_col = False, encoding = "ISO-8859-1" )
line = df['Mentions']
st = open('konjungsi.csv', 'r')
stopWords = getStopWordList('konjungsi.csv')
result_list = []

In [12]:
for x in range(0, len(line)):
    processedTweet = processTweet(line[x])
#     print(processedTweet)
    featureVector = getFeatureVector(processedTweet)
#     print(featureVector)
    replaceAbb = translator(featureVector)
#     print(replaceAbb)
    result_list.append(replaceAbb)   
#     result_list.append(temp)
#     line = fp.readline()
#end loop
print(result_list)

[['baca', 'sejarah', 'kenapa', 'privatisasi', 'bumn'], ['ancaman', 'gugatan', 'dilayangkan', 'partai', 'berkarya', 'jurubicara', 'tkn', 'ahmad', 'basarah', 'memancing', 'kemarahan'], ['presiden', 'jokowi', 'resmikan', 'tol', 'bocimi', 'seksi', 'i'], ['kalau', 'orang', 'rajin', 'sholat', 'puasa', 'trus', 'rajin', 'ngibulin', 'orang', 'bagaimana', 'tu', 'bro'], ['deklarasi', 'dukung', 'insya', 'allah', 'diumumkan', 'januari'], ['habib', 'bahar', 'enggan', 'maaf', 'presiden', 'jokowi'], ['enak', 'zaman'], ['dana', 'desa', 'meningkatkan', 'infrastruktur', 'desa', 'desa', 'mandiri', 'dan', 'sejahtera', 'jokowimembangundesa'], ['pasukan', 'masuk', 'monas', 'pasukan', 'guru', 'honorer', 'pasukan', 'ojol', 'lengkap', 'dengan', 'seragamnya', 'goodbye'], [], ['part'], ['dana', 'desa', 'meningkatkan', 'infrastruktur', 'desa', 'desa', 'mandiri', 'dan', 'sejahtera', 'jokowimembangundesa'], ['ancaman', 'politik', 'jokowi', 'semangatnya', 'persatuan'], ['reuniakbar212dimonas', 'kampanye', 'prabowo', 

In [13]:
tempStr = []
for baris in range(0, len(result_list)):
    tempo=""
    for kolom in range(0,len(result_list[baris])):
        tempo = tempo+" "+result_list[baris][kolom]
    tempStr.append(tempo)
# print(type(tempStr))

In [14]:
tempData = df['Mentions']
for x in range(0,len(tempStr)):
    df['Mentions'][x] = tempStr[x]
#     tempData.replace(x,tempStr[x])

print(df['Mentions'])


0                   baca sejarah kenapa privatisasi bumn
1       ancaman gugatan dilayangkan partai berkarya j...
2            presiden jokowi resmikan tol bocimi seksi i
3       kalau orang rajin sholat puasa trus rajin ngi...
4         deklarasi dukung insya allah diumumkan januari
5                habib bahar enggan maaf presiden jokowi
6                                             enak zaman
7       dana desa meningkatkan infrastruktur desa des...
8       pasukan masuk monas pasukan guru honorer pasu...
9                                                       
10                                                  part
11      dana desa meningkatkan infrastruktur desa des...
12          ancaman politik jokowi semangatnya persatuan
13      reuniakbar212dimonas kampanye prabowo memakai...
14      account oposisi pemerintah block dukung jokow...
15                 bangga polri penuh cinta kasih sayang
16      prabowo subianto didukung anak cucu jendral s...
17                             

In [20]:
# mentions = df['Mentions']
# tipes = df['Type']
# df['Date'] = df['Date']
# dates = df['Date']
# medias = df['Media']
# sentiments = df['Sentiment']
# # influencers = df['Influencer']
# coba2 = []
# ahai = []
# # print(dates[0])
# for i in range(0,len(dates)):
#     ahai = datetime.strptime(dates[i], '%m/%d/%Y %H:%M')
#     coba2.append(ahai)
# ahai = datetime.strptime("12/01/2018 23:59", '%m/%d/%Y %H:%M')
# print(datetime.strptime("12/01/2018 23:59", '%m/%d/%Y %H:%M'))
# print(coba2)
#     coba[i] = datetime.strptime(dates[i], '%m/%w/%Y %H:%M')
    
# print(dates[1])
# Thu Jan 03 09:38:31 +0000 2019
# print(coba)
# print(datetime.strptime("Thu Jan 03 09:38:31 +0000 2019", '%a %b %d %H:%M:%S +0000 %Y'))
# print(datetime.strptime(dates[0], '%m/%w/%Y %H:%M'))
# tempData = df['Mentions']
# es = Elasticsearch()

# for index, row in df.iterrows():
#     es.index(index="logstash-a",
#                          # create/inject data into the cluster with index as 'logstash-a'
#                          # create the naming pattern in Management/Kinaba later in order to push the data to a dashboard
#                          doc_type="test-type",
#                          body={
#                                 "author": row['Influencer'],
#                                #parse the milliscond since epoch to elasticsearch and reformat into datatime stamp in Kibana later
#                                "date": datetime.strptime(row['Date'], '%m/%d/%Y %H:%M'),
#                                "message":row['Mentions'],
#                                "media":row['Media'],
#                                "sentiment": row['Sentiment']})

In [41]:
print(coba2)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [60]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df['text'], 
                                                    df['sentiment'],
                                                    random_state=0)

In [61]:
X_test.index

Int64Index([ 18, 170, 107,  98, 177, 182,   5, 146,  12, 152,  61, 125, 180,
            154,  80,   7,  33, 130,  37,  74, 183, 145,  45, 159,  60, 123,
            179, 185, 122,  44,  16,  55, 150, 111,  22, 189, 129,   4,  83,
            106, 134,  66,  26, 113, 168,  63,   8,  75, 118, 143],
           dtype='int64')

In [62]:
print(len(y_train), len(y_test))

150 50


In [78]:
print('X_train first entry:\n\n', X_train.iloc[3])
print('\n\nX_train shape: ', X_train.shape)

X_train first entry:

  versi ketua gp bendera berceceran diselematkan bendera tsb jatuh tidak semestinya bendera baik dibakar bakar dilipat simpan eh alasan bendera hti


X_train shape:  (150,)


In [66]:
vect = TfidfVectorizer(min_df=5, stop_words='english', use_idf=True, ngram_range=(1,2)).fit(X_train)
len(vect.get_feature_names())

226

In [67]:
X_train_vectorized = vect.transform(X_train)

In [68]:
mnb = MultinomialNB()

mnb.fit(X_train_vectorized, y_train)

predictions = mnb.predict(vect.transform(X_test))

print('AUC: ', roc_auc_score(y_test, predictions))
print('Confusion Metrix: ', confusion_matrix(y_test,predictions))
print('ACC: ', accuracy_score(y_test, predictions))

AUC:  0.9615384615384616
Confusion Metrix:  [[24  0]
 [ 2 24]]
ACC:  0.96


In [80]:
print(mnb.predict(vect.transform([' permintaan maaf saran gus solah proses hukum ngomong idiot tersangka tebang pilih pembakaran bendera tauhid melukai hati umat cepat ratna sarumpaet adillah'])))

[0]
